## Notebook to test the vae model on smaller dataset

### load and sample the data

In [1]:

import sys

sys.path.append("../../")

import utils.load_csv as load_csv

set = load_csv.load_pandas()
set = load_csv.sample_data(set, 100)

# train and val only contain inliers and test contains both inliers and outliers
train, val, test = load_csv.split_data(set)
print(f"train length: {len(train)}")
print(f"val length: {len(val)}")
print(f"test length: {len(test)}")

print(f'outlier test training: {any(train["classification"] != 1)}')
print(f'outlier test validation: {any(val["classification"] != 1)}')
print(f'outlier test test: {any(test["classification"] != 1)}')
train

/home/bendm/machine_learning/project/majority-vote-outlier-detection/models/VAE/../../utils/load_csv.py:20: DtypeWarning: Columns (91,94,209,213) have mixed types. Specify dtype option on import or set low_memory=False.
  frame = pd.read_csv(data_path, delimiter=";").iloc[:, -3:]


train length: 45
val length: 15
test length: 18
outlier test training: False
outlier test validation: False
outlier test test: True


,classification,file_names,label
2894,1,/home/bendm/machine_learning/project/majority-...,Singlet
2876,1,/home/bendm/machine_learning/project/majority-...,Singlet
4947,1,/home/bendm/machine_learning/project/majority-...,Singlet
5861,1,/home/bendm/machine_learning/project/majority-...,Singlet
2678,1,/home/bendm/machine_learning/project/majority-...,Singlet
7231,1,/home/bendm/machine_learning/project/majority-...,Singlet
9930,1,/home/bendm/machine_learning/project/majority-...,Singlet
971,1,/home/bendm/machine_learning/project/majority-...,Singlet
6340,1,/home/bendm/machine_learning/project/majority-...,Singlet
4640,1,/home/bendm/machine_learning/project/majority-...,Singlet


In [2]:
from models.VAE.VAE import VAE
from datetime import datetime

import torch
from torch.utils.tensorboard import SummaryWriter
from utils.dataloader import ImagePathDataset
from torchvision.transforms import v2

In [3]:
batch_size = 8
lr = 1e-3
weight_decay = 1e-2
epochs = 80
input_dim = 4800
hidden_dim = 2000
latent_dim = 30

transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
        v2.Resize((60, 80)),
        v2.ToTensor(),
        v2.Lambda(
            lambda x: (x.view(-1) - torch.min(x)) / (torch.max(x) - torch.min(x))
        ),
    ]
)

train_set = ImagePathDataset(train, transform=transform)
val_set = ImagePathDataset(val, transform=transform)

train_dataloader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True
)
val_dataloader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, shuffle=True
)

/home/bendm/machine_learning/project/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
import matplotlib.pyplot as plt
import numpy as np

for batch_idx, (data, target) in enumerate(train_dataloader):
    # check if all values are 0
    if ((data == 0).all()):
        print("nan")
        print(data)
        print(data.shape)
        plt.imshow(data[0].reshape(60, 80))
        plt.show()
        break

reasonable output after all preprocessing transformations.

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(input_dim=input_dim, hidden_dim=hidden_dim, latent_dim=latent_dim).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
writer = SummaryWriter(f'runs/outlier/vae_{datetime.now().strftime("%Y%m%d-%H%M%S")}')

In [ ]:
prev_updates = 0
for epoch in range(epochs):
    print(f"Epoch {epoch+1}/{epochs}")
    prev_updates = model.train_model(train_dataloader, optimizer, prev_updates, writer)
    model.test(val_dataloader, prev_updates, writer)

Epoch 1/80


  0%|          | 0/6 [00:00<?, ?it/s]

Step 0 (N samples: 0), Loss: 3330.5256 (Recon: 3327.2664, KL: 3.2593) Grad: 28.0864


Testing: 100%|██████████| 2/2 [00:00<00:00, 18.59it/s]


====> Test set loss: 2605.7109 (BCE: 2468.2310, KLD: 137.4800)
Epoch 2/80


...

Testing: 100%|██████████| 2/2 [00:00<00:00, 26.62it/s]


====> Test set loss: 1999.5583 (BCE: 1997.0443, KLD: 2.5140)
Epoch 19/80


Testing: 100%|██████████| 2/2 [00:00<00:00, 26.23it/s]


====> Test set loss: 1994.1074 (BCE: 1989.9628, KLD: 4.1446)
Epoch 20/80


 17%|█▋        | 1/6 [00:00<00:01,  3.35it/s]